In [143]:
import pickle
import re 
import pandas as pd
import re 
import datetime as dt

In [144]:
import locale 
locale.setlocale(locale.LC_TIME, 'da_DK.UTF-8') 

'da_DK.UTF-8'

In [186]:
def baseinfo_to_df(data):
    
    baseinfos = []

    baseinfo_keys = ['Adresse', 'Boligstørrelse', 'Grundstørrelse', 'Værelser', 'Etage', 'Byggeår', 'Energimærke', 'Ejerudgift', 'Kælderstørrelse', 'Pris', 'Boligtype', 'Oprettet', 'Markedstid', 'Link', 'Link_alt', 'boliga_link']
    
    for bolig, time in data:
        baseinfo = { b: bolig[b].replace('&nbsp;', ' ')\
                                 .replace('m²', '')\
                                 .replace('. sal', '')\
                                 .replace('  kr. / md.', '')\
                                 .strip()
                     for b in bolig if b in baseinfo_keys }


        def markedstid_dage(markedstid_tekst):
            antal = int(re.search('^\d*', markedstid_tekst).group())
            if 'måned' in markedstid_tekst:
                antal *= 30
            elif 'år' in markedstid_tekst:
                antal *= 365
            else:
                pass
            return antal

        #print(baseinfo.keys())
        if len(baseinfo.keys()) < 3:
            break

        baseinfo['Markedstid_dage'] = markedstid_dage(baseinfo['Markedstid'])

        baseinfo['Pris'] = baseinfo['Pris'].replace('kr.', '').replace('.', '').strip()
        baseinfo['Ejerudgift'] = baseinfo['Ejerudgift'].replace('kr.', '').replace('.', '').strip()

        baseinfos.append(baseinfo)

    df = pd.DataFrame(baseinfos)
    return df 

def timeline_to_df(data):
    
    timelineinfos = []
    df_temp = pd.DataFrame()
    
    timeline_keys = ['Timeline_time', 'Timeline_action', 'Timeline_price']

    for bolig, time in data:
        if len(bolig.keys()) < 3:
            break
        #try:
        adresse = bolig['Adresse']
        #except:
        #    print(bolig)
        boliga_link = bolig['boliga_link']

        timelineinfo = {}
        
        # Since the timelines are repeating if less than 10 observation, we will have to cut the data once it starts repeats. Data is cyclically orderes by time 
        rows_to_keep = 0
        earliest_data = dt.datetime.strptime('jan 2050', '%b %Y')
    
        for timeline_key in [key for key in bolig if key in timeline_keys]:
            #print(bolig[timeline_key])
            values = []
            
            for time_value in bolig[timeline_key]:
                
                
                #timelineinfo = {}
                #timelineinfo['Adresse'] = adresse
                #timelineinfo['boliga_link'] = boliga_link
                #timelineinfo['Variable'] = timeline_key
                value_temp = re.sub('<.*>', '', time_value.replace('&nbsp;', ' ').replace('kr.', '').replace('.', ''))
                if timeline_key == 'Timeline_time':
                    value_temp = value_temp.title()
                    #print(value_temp)
                    value_temp = dt.datetime.strptime(value_temp, '%b %Y')
                    if value_temp < earliest_data:
                        #print(value_temp, earliest_data)
                        earliest_data = value_temp
                        rows_to_keep += 1
                values.append(value_temp)
                #timelineinfo['Value'] = value_temp
                #print(timeline_key,value_temp)
                #timelineinfos.append(timelineinfo)
            timelineinfo[timeline_key] = values
        
        for timeline_key in timelineinfo:
            timelineinfo[timeline_key] =  timelineinfo[timeline_key][:rows_to_keep]
        # Add identifier
        timelineinfo['Adresse'] = [adresse]*len(timelineinfo[timeline_key])
        timelineinfo['boliga_link'] = [boliga_link]*len(timelineinfo[timeline_key])
        #print('rows_to_keep', rows_to_keep)
        #timelineinfos.append(timelineinfo)
        #print(pd.DataFrame(timelineinfo))
        df_temp = pd.concat([df_temp, pd.DataFrame(timelineinfo)], axis=0)
        #df_temp.append(pd.DataFrame(timelineinfo))
    #print(timelineinfos)
    df = df_temp # pd.DataFrame(timelineinfos)
    return df 
    


In [198]:
results_path = []

df_baseinfo = pd.DataFrame()
df_timeline = pd.DataFrame()

for i in range(1,48):
    print(i, flush=True)
    results_path = f'data/bolig_data/bolig_data_{i}.pickle'
    data = pickle.load(open(results_path, 'rb'))
    df_baseinfo_temp = baseinfo_to_df(data)
    df_timeline_temp = timeline_to_df(data)

    df_baseinfo = pd.concat([df_baseinfo, df_baseinfo_temp], axis=0)
    df_timeline = pd.concat([df_timeline, df_timeline_temp], axis=0)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [199]:
# print('no data', len(data))
# for bolig, time in data:
#     sep_len = 25
#     for k in bolig:
#         if isinstance(bolig[k], list):
#             print(k + ':')
#             for l in bolig[k]:
#                 print(' '*sep_len, re.sub('<.*>', '', l.replace('&nbsp;', ' ')).strip())
#         else:
#             print(k +':', ' '*(sep_len-len(k)-2), bolig[k].replace('&nbsp;', ' '))
#     print('')
#     print('-'*50)
#     print('')


In [200]:
df_baseinfo[['Boligstørrelse', 'Grundstørrelse', 'Værelser', 'Etage', 'Byggeår', 'Pris', 'Ejerudgift', 'Kælderstørrelse']] = df_baseinfo[['Boligstørrelse', 'Grundstørrelse', 'Værelser', 'Etage', 'Byggeår', 'Pris', 'Ejerudgift', 'Kælderstørrelse']].astype(float)

In [201]:
df_baseinfo.dtypes

Adresse             object
Boligstørrelse     float64
Grundstørrelse     float64
Værelser           float64
Etage              float64
Byggeår            float64
Energimærke         object
Ejerudgift         float64
Kælderstørrelse    float64
Pris               float64
Boligtype           object
Oprettet            object
Markedstid          object
Link                object
Link_alt            object
boliga_link         object
Markedstid_dage      int64
dtype: object

In [203]:
df_baseinfo.shape

(2355, 17)

In [204]:
df_baseinfo.describe()

,Boligstørrelse,Grundstørrelse,Værelser,Etage,Byggeår,Ejerudgift,Kælderstørrelse,Pris,Markedstid_dage
count,2354.000000,2354.000000,2354.000000,2355.000000,2354.000000,2355.000000,2354.000000,2.355000e+03,2355.000000
mean,100.368309,76.081392,3.370858,2.264544,1959.966440,4327.242887,6.741716,4.724374e+06,118.866242
std,44.358916,203.453305,1.411091,3.036747,106.960201,2047.191583,23.453918,2.958735e+06,134.493649
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.686430e+05,0.000000
25%,68.000000,0.000000,2.000000,0.000000,1931.000000,2975.500000,0.000000,2.795000e+06,23.000000
50%,95.000000,0.000000,3.000000,1.000000,1970.000000,3997.000000,0.000000,4.195000e+06,60.000000
75%,120.000000,0.000000,4.000000,3.000000,2019.000000,5111.500000,0.000000,5.747500e+06,180.000000
max,412.000000,994.000000,12.000000,29.000000,2022.000000,19794.000000,183.000000,3.000000e+07,1825.000000


In [205]:
df_baseinfo

,Adresse,Boligstørrelse,Grundstørrelse,Værelser,Etage,Byggeår,Energimærke,Ejerudgift,Kælderstørrelse,Pris,Boligtype,Oprettet,Markedstid,Link,Link_alt,boliga_link,Markedstid_dage
0,"Theklavej 39 st. 47., 2400 København NV",94.0,0.0,3.0,0.0,1974.0,D,4779.0,0.0,2695000.0,Ejerlejlighed,Pris er faldet,27 dage på markedet,https://danbolig.dk/bolig/koebenhavn/2400/ejer...,https://danbolig.dk/bolig/koebenhavn/2400/ejer...,https://www.boliga.dk/bolig/1638173/theklavej_...,27
1,"Håbets Allé 7 1. th., 2700 Brønshøj",81.0,0.0,3.0,1.0,1936.0,D,3931.0,0.0,2500000.0,Ejerlejlighed,Pris er faldet,1 måned på markedet,https://danbolig.dk/bolig/koebenhavn/2700/ejer...,https://danbolig.dk/bolig/koebenhavn/2700/ejer...,https://www.boliga.dk/bolig/1636808/haabets_al...,30
2,"Kongens Nytorv 21, 3. tv., 1050 København K",280.0,0.0,7.0,3.0,1767.0,-,19794.0,0.0,15995000.0,Ejerlejlighed,Pris er faldet,5 måneder på markedet,NaN,https://home.dk/boligkatalog/koebenhavn/1050/e...,https://www.boliga.dk/bolig/1603678/kongens_ny...,150
3,"Nyhavn 38, st. Dør/lejl. 4, 1051 København K",83.0,0.0,4.0,0.0,1880.0,C,5702.0,88.0,7875000.0,Ejerlejlighed,Pris er faldet,9 måneder på markedet,NaN,https://www.edc.dk/alle-boliger/k%C3%B8benhavn...,https://www.boliga.dk/bolig/1567291/nyhavn_38_...,270
4,"Nyhavn 23, 4., 1051 København K",166.0,0.0,4.0,4.0,1700.0,-,4138.0,0.0,8495000.0,Ejerlejlighed,Pris er faldet,5 måneder på markedet,NaN,https://www.nybolig.dk/ejerlejlighed/1051/nyha...,https://www.boliga.dk/bolig/1602025/nyhavn_23_...,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,"Tuborgvej 19, 1. Dør/lejl. 8, 2900 Hellerup",80.0,0.0,3.0,1.0,1978.0,C,4757.0,0.0,3600000.0,Ejerlejlighed,Pris er faldet,5 måneder på markedet,NaN,https://www.edc.dk/alle-boliger/hellerup/2900/...,https://www.boliga.dk/bolig/1599061/tuborgvej_...,150
48,"Esthersvej 22B 2. TH., 2900 Hellerup",83.0,0.0,3.0,2.0,1933.0,D,3926.0,0.0,3795000.0,Ejerlejlighed,Pris er faldet,25 dage på markedet,NaN,https://danbolig.dk/bolig/koebenhavn/2900/ejer...,https://www.boliga.dk/bolig/1638466/esthersvej...,25
49,"Judithsvej 7, st. tv, 2900 Hellerup",67.0,0.0,3.0,0.0,1932.0,C,2998.0,0.0,3895000.0,Ejerlejlighed,Pris er faldet,5 måneder på markedet,NaN,https://www.nybolig.dk/ejerlejlighed/2900/judi...,https://www.boliga.dk/bolig/1602590/judithsvej...,150
50,"Esthersvej 5, 4. tv, 2900 Hellerup",77.0,0.0,3.0,4.0,2004.0,C,5318.0,6.0,3995000.0,Ejerlejlighed,Pris er faldet,11 måneder på markedet,NaN,https://www.edc.dk/alle-boliger/hellerup/2900/...,https://www.boliga.dk/bolig/1547454/esthersvej...,330


In [230]:
import numpy as np

In [234]:
df_baseinfo['kvm_pris'] = (df_baseinfo['Pris'] / df_baseinfo['Boligstørrelse']).fillna(0).replace(np.inf, 0)
#df_baseinfo['kvm_pris'].sort_values().values[-1]

In [240]:
pd.set_option('display.max_colwidth', -1)
df_baseinfo[(df_baseinfo['Boligstørrelse'] > 50) & 
            (df_baseinfo['Boligstørrelse'] < 80) &
            (df_baseinfo['Værelser'] >= 2) & 
            (df_baseinfo['Pris'] <= 2_000_000) &
           (df_baseinfo['Boligtype'] != 'Andelsbolig')
           ].sort_values('Boligstørrelse', ascending=False).head(50)# .columns # .sort_values('kvm_pris')

<ipython-input-240-93de3d378db7>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Adresse,Boligstørrelse,Grundstørrelse,Værelser,Etage,Byggeår,Energimærke,Ejerudgift,Kælderstørrelse,Pris,Boligtype,Oprettet,Markedstid,Link,Link_alt,boliga_link,Markedstid_dage,kvm_pris
14,"Vigerslevvej 278A, 2. th, 2500 Valby",73.0,0.000,3.0,2.0,1936.0,E,4612.0,0.0,1998000.0,Ejerlejlighed,Pris er faldet,1 måned på markedet,NaN,https://www.nybolig.dk/ejerlejlighed/2500/vigerslevvej/260349/320047,https://www.boliga.dk/bolig/1636443/vigerslevvej_278a_2_th_2500_valby,30,27369.863014
26,"Frederikssundsvej 414 st. th., 2700 Brønshøj",71.0,0.000,2.0,0.0,1954.0,D,3007.0,0.0,1995000.0,Ejerlejlighed,Pris er faldet,2 måneder på markedet,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://www.boliga.dk/bolig/1624303/frederikssundsvej_414__st_th_2700_broenshoej,60,28098.591549
19,"Islevhusvej 35, 2., 2700 Brønshøj",71.0,0.000,4.0,2.0,1938.0,F,2198.0,0.0,1795000.0,Villalejlighed,Pris er faldet,6 måneder på markedet,NaN,https://home.dk/boligkatalog/koebenhavn/2700/villalejligheder/islevhusvej_35_2_1020000104.aspx,https://www.boliga.dk/bolig/1597598/islevhusvej_35_2_2700_broenshoej,180,25281.690141
1,"Frederikssundsvej 414 st. th., 2700 Brønshøj",71.0,0.000,2.0,0.0,1954.0,D,3007.0,0.0,1995000.0,Ejerlejlighed,Pris er faldet,2 måneder på markedet,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://www.boliga.dk/bolig/1624303/frederikssundsvej_414__st_th_2700_broenshoej,60,28098.591549
0,"Frederikssundsvej 414 st. th., 2700 Brønshøj",71.0,0.000,2.0,0.0,1954.0,D,3007.0,0.0,1995000.0,Ejerlejlighed,Pris er faldet,2 måneder på markedet,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://danbolig.dk/bolig/koebenhavn/2700/ejerlejlighed/063ekh02041-063,https://www.boliga.dk/bolig/1624303/frederikssundsvej_414__st_th_2700_broenshoej,60,28098.591549
25,"Amagerbrogade 263A, 2. th., 2300 København S",69.0,0.000,3.0,2.0,1938.0,D,3142.0,0.0,1945000.0,Ejerlejlighed,Pris er faldet,7 måneder på markedet,NaN,https://home.dk/boligkatalog/koebenhavn/2300/ejerlejligheder/amagerbrogade_263a_2_th_1340000065.aspx,https://www.boliga.dk/bolig/1586276/amagerbrogade_263a_2_th_2300_koebenhavn_s,210,28188.405797
32,"Frederiksborgvej 199, 2., 2400 København NV",67.0,0.000,3.0,2.0,1933.0,D,2331.0,0.0,1995000.0,Ejerlejlighed,Pris er faldet,3 måneder på markedet,NaN,https://www.estate.dk/ejerlejlighed/2400/frederiksborgvej/270172/200156,https://www.boliga.dk/bolig/1617074/frederiksborgvej_199_2_2400_koebenhavn_nv,90,29776.119403
40,"Fossgårdsvej 8 1, 2720 Vanløse",67.0,382.000,2.0,1.0,1941.0,E,2356.0,70.0,1800000.0,Villalejlighed,Pris er faldet,9 måneder på markedet,NaN,https://www.nybolig.dk/ejerlejlighed/2720/fossgaardsvej/260385/bw1767,https://www.boliga.dk/bolig/1564486/fossgaardsvej_8_1_2720_vanloese,270,26865.671642
25,"Birkevang 105, 2700 Brønshøj",67.0,332.000,3.0,0.0,1962.0,C,2961.0,0.0,1895000.0,Villa,Pris er faldet,5 dage på markedet,NaN,https://www.estate.dk/villa/2700/birkevang/260146/507104,https://www.boliga.dk/bolig/1645072/birkevang_105_2700_broenshoej,5,28283.582090
2,"Sallingvej 66, st.. tv., 2720 Vanløse",67.0,0.000,2.0,0.0,1941.0,D,3897.0,0.0,1995000.0,Ejerlejlighed,Pris er faldet,1 måned på markedet,NaN,https://www.lokalbolig.dk?sag=17-X0000393&amp;mgl=3532,https://www.boliga.dk/bolig/1635398/sallingvej_66_st_tv_2720_vanloese,30,29776.119403
